In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import pickle

MoveNet 모델 로드

In [2]:
model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
movenet = model.signatures['serving_default']

이미지에서 포즈 키포인트를 추출하는 함수

In [3]:
def run_inference(movenet, image):
    """이미지에서 포즈 키포인트를 추출하는 함수"""
    input_image = tf.image.resize_with_pad(tf.expand_dims(image, axis=0), 192, 192)
    input_image = tf.cast(input_image, dtype=tf.int32)
    
    # Run model inference
    keypoints_with_scores = movenet(input_image)
    return keypoints_with_scores['output_0'].numpy()

동영상에서 프레임별로 포즈 키포인트를 추출하는 함수

In [4]:
def extract_keypoints_from_video_frames(video_path, movenet):
    cap = cv2.VideoCapture(video_path)
    keypoints_list = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 현재 프레임에서 MoveNet을 사용하여 키포인트 추출
        keypoints = run_inference(movenet, frame)
        keypoints_list.append(keypoints)
    
    cap.release()
    return keypoints_list

실시간 증강 - 좌우 반전, 시간적 증강(2,3)을 수행한 함수

In [33]:
def extract_and_augment_keypoints(video_path, movenet, sampling_rate=2, additional_sampling_rate=3):
    cap = cv2.VideoCapture(video_path)
    original_keypoints = []
    flipped_keypoints = []
    sampled_keypoints = []  # sampling_rate=2에 대한 키포인트
    additional_sampled_keypoints = []  # 추가적인 sampling_rate=3에 대한 키포인트
    additional_sampled_keypoints_2 = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 원본 프레임에서 키포인트 추출
        keypoints = run_inference(movenet, frame)
        original_keypoints.append(keypoints)

        # 프레임을 좌우 반전하여 키포인트 추출
        frame_flipped = cv2.flip(frame, 1)
        keypoints_flipped = run_inference(movenet, frame_flipped)
        flipped_keypoints.append(keypoints_flipped)

        # 기존 시간적 증강: sampling_rate=2에 따라 키포인트 추출
        if frame_count % sampling_rate == 0:
            sampled_keypoints.append(keypoints)
        
        # 추가적인 시간적 증강: additional_sampling_rate=3에 따라 키포인트 추출
        if frame_count % additional_sampling_rate == 0:
            additional_sampled_keypoints.append(keypoints)
            
        # 추가적인 시간적 증강: additional_sampling_rate=3에 따라 키포인트 추출
        if frame_count % 4 == 0:
            additional_sampled_keypoints_2.append(keypoints)

        frame_count += 1
    
    cap.release()
    # 모든 키포인트 리스트와 추가적인 샘플링된 키포인트 리스트를 반환합니다.
    return original_keypoints, flipped_keypoints, sampled_keypoints, additional_sampled_keypoints, additional_sampled_keypoints_2

In [34]:
# 아래는 데이터 로딩 및 처리의 예시 코드입니다.
video_root = '/Users/diana/Downloads/BabyPose-main/data'
video_dirs = ['arching_back', 'head_banging', 'kicking_legs', 'rubbing_eye', 'stretching', 'sucking_fingers']

keypoints_list = []
labels = []

# 데이터 로딩 및 처리
for label, class_dir in enumerate(video_dirs):
    class_path = os.path.join(video_root, class_dir)
    video_files = [f for f in os.listdir(class_path) if f.endswith('.mp4')]

    for video_file in tqdm.tqdm(video_files, desc=f'Processing {class_dir}'):
        video_path = os.path.join(class_path, video_file)
        
        # 원본, 좌우 반전, 시간적 증강된 동영상에서 키포인트 추출
        original_kp, flipped_kp, sampled_kp, additional_sampled_kp, additional_sampled_kp_2 = extract_and_augment_keypoints(video_path, movenet)
        
        keypoints_list.append(original_kp)
        labels.append(label)  # 원본 데이터에 대한 레이블 추가

        keypoints_list.append(flipped_kp)
        labels.append(label)  # 좌우 반전된 데이터에 대한 레이블 추가

        keypoints_list.append(sampled_kp)
        labels.append(label)  # 시간적 증강된 데이터 - 2 에 대한 레이블 추가
        
        keypoints_list.append(additional_sampled_kp)
        labels.append(label)  # 시간적 증강된 데이터 - 3 에 대한 레이블 추가
        
        keypoints_list.append(additional_sampled_kp_2)
        labels.append(label)  # 시간적 증강된 데이터 - 4 에 대한 레이블 추가

# 데이터 저장
with open('keypoints_data_augmented.pkl', 'wb') as f:
    pickle.dump({'keypoints': keypoints_list, 'labels': labels}, f)

print("Finished processing and saving augmented data.")

Processing sucking_fingers: 100%|██████████| 32/32 [03:03<00:00,  5.73s/it]


Finished processing and saving augmented data.


각 동영상 별로 평균 좌표, 신뢰도를 구하는 함수

In [11]:
def calculate_mean_keypoints_from_file(keypoints_data):
    
    # 모든 동영상에 대한 평균 키포인트 계산
    mean_keypoints_all_videos = []
    for keypoints_list in keypoints_data['keypoints']:
        # 각 동영상에 대한 키포인트 리스트에서 평균 계산
        mean_keypoints = [[sum(pos) / len(keypoints_list) for pos in zip(*frame)] for frame in zip(*keypoints_list)]
        mean_keypoints_all_videos.append(mean_keypoints)
    
    return mean_keypoints_all_videos, keypoints_data['labels']

키포인트 변화량 계산하는 함수

In [12]:
def calculate_keypoint_changes(keypoints_data):
    # 변경된 부분: 이미 로드된 키포인트 데이터를 직접 사용
    # 키포인트 데이터는 각 동영상의 프레임별 키포인트 리스트를 포함하는 리스트

    changes_list = []  # 변화량을 저장할 리스트 초기화

    for keypoints_list in keypoints_data['keypoints']:
        changes = []  # 개별 동영상의 키포인트 변화량을 저장할 리스트
        prev_keypoints = None

        for keypoints in keypoints_list:
            keypoints = np.array(keypoints)
            if prev_keypoints is not None:
                # 현재 프레임과 이전 프레임의 키포인트 사이의 변화량 계산
                change = np.abs(keypoints - prev_keypoints)
                changes.append(change)
            prev_keypoints = keypoints

        # 모든 변화량의 평균 계산
        if changes:
            mean_changes = np.mean(changes, axis=0)
        else:
            # 변화량이 없는 경우, 0으로 채워진 배열 반환
            mean_changes = np.zeros_like(keypoints_list[0])

        changes_list.append(mean_changes)

    return changes_list

키포인트 각도 변화량 계산하는 함수-> 이거 3개 키포인트만 되니까 유의미한 키포인트 추출해서 쓰도록 조정해보기

In [13]:
def calculate_angle(point1, point2, point3):
    """
    세 점을 이용하여 두 벡터 사이의 각도를 계산합니다.
    :param point1, point2, point3: 각 점의 좌표를 나타내는 (x, y) 튜플이나 리스트.
    :return: 두 벡터 사이의 각도(도).
    """
    # 벡터 v1과 v2 생성
    v1 = np.array(point1) - np.array(point2)
    v2 = np.array(point3) - np.array(point2)
    
    # 벡터의 내적과 노름(크기)을 사용하여 각도(라디안) 계산
    angle_rad = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    
    # 각도를 도로 변환
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

# 평균
def calculate_angle_changes(keypoints_data, point_indices):
    angle_changes_list = []
    for keypoints_list in keypoints_data['keypoints']:
        angles = []
        for frame_keypoints in keypoints_list:
            # 키포인트 데이터가 충분한지 확인
            if len(frame_keypoints) > max(point_indices):
                p1 = frame_keypoints[point_indices[0]][:2]  # x, y 좌표만 사용
                p2 = frame_keypoints[point_indices[1]][:2]
                p3 = frame_keypoints[point_indices[2]][:2]
                angle = calculate_angle(p1, p2, p3)
                angles.append(angle)
            else:
                # 충분한 데이터가 없는 경우 계산에서 제외
                continue
        
        if angles:  # 각도 데이터가 있을 경우에만 계산
            angle_changes = np.abs(np.diff(angles))
            mean_angle_change = np.mean(angle_changes)
            angle_changes_list.append(mean_angle_change)
        else:
            # 각도 데이터가 없는 경우 0으로 처리
            angle_changes_list.append(0)
    
    return np.array(angle_changes_list)

# 최솟값
def calculate_min_angle_changes(keypoints_data, point_indices):
    min_angle_changes_list = []
    for keypoints_list in keypoints_data['keypoints']:
        angles = []
        for frame_keypoints in keypoints_list:
            if len(frame_keypoints) > max(point_indices):
                p1 = frame_keypoints[point_indices[0]][:2]  # x, y 좌표만 사용
                p2 = frame_keypoints[point_indices[1]][:2]
                p3 = frame_keypoints[point_indices[2]][:2]
                angle = calculate_angle(p1, p2, p3)
                angles.append(angle)

        if angles:
            angle_changes = np.abs(np.diff(angles))
            min_angle_change = np.min(angle_changes) if len(angle_changes) > 0 else 0
            min_angle_changes_list.append(min_angle_change)
        else:
            min_angle_changes_list.append(0)
    
    return np.array(min_angle_changes_list)

# 최댓값
def calculate_max_angle_changes(keypoints_data, point_indices):
    max_angle_changes_list = []
    for keypoints_list in keypoints_data['keypoints']:
        angles = []
        for frame_keypoints in keypoints_list:
            if len(frame_keypoints) > max(point_indices):
                p1 = frame_keypoints[point_indices[0]][:2]  # x, y 좌표만 사용
                p2 = frame_keypoints[point_indices[1]][:2]
                p3 = frame_keypoints[point_indices[2]][:2]
                angle = calculate_angle(p1, p2, p3)
                angles.append(angle)

        if angles:
            angle_changes = np.abs(np.diff(angles))
            max_angle_change = np.max(angle_changes) if len(angle_changes) > 0 else 0
            max_angle_changes_list.append(max_angle_change)
        else:
            max_angle_changes_list.append(0)
    
    return np.array(max_angle_changes_list)


움직임 패턴의 자기상관성을 계산하는 함수:
평균 자기상관성, 
자기상관성의 표준편차,
피크 수(평균 자기상관성 이상의 값을 가지는 피크의 수를 계산합니다. 이는 반복되는 패턴의 빈도를 나타낼 수 있습니다.)

In [14]:
def calculate_enhanced_autocorrelation_features(keypoints_data):
    features_list = []  # 각 동영상의 향상된 자기상관성 특성을 저장할 리스트

    for keypoints_list in keypoints_data['keypoints']:
        changes = []
        prev_keypoints = None
        for keypoints in keypoints_list:
            keypoints = np.array(keypoints)
            if prev_keypoints is not None:
                change = np.linalg.norm(keypoints - prev_keypoints)
                changes.append(change)
            prev_keypoints = keypoints

        if changes:
            changes = np.array(changes)
            autocorrelation = np.correlate(changes - np.mean(changes), changes - np.mean(changes), mode='full')
            autocorrelation = autocorrelation[autocorrelation.size // 2:]  # 자기상관성 값 중 양의 지연만 고려

            # 향상된 특성 계산
            mean_autocorrelation = np.mean(autocorrelation)
            std_autocorrelation = np.std(autocorrelation)
            peak_count = np.sum(autocorrelation > (mean_autocorrelation + std_autocorrelation))  # 평균 이상의 피크 수

            features = [mean_autocorrelation, std_autocorrelation, peak_count]
        else:
            features = [0, 0, 0]

        features_list.append(features)

    return features_list

pickle 파일 로드

In [35]:
pkl_file_path = '/Users/diana/Desktop/BabyposeModel/capstone2_SEDA/keypoints_data_augmented.pkl'

# pickle 파일 로드
with open(pkl_file_path, 'rb') as f:
     keypoints_data = pickle.load(f)

feature 구성

In [36]:
mean_keypoints_all_videos, labels = calculate_mean_keypoints_from_file(keypoints_data)
changes_list = calculate_keypoint_changes(keypoints_data)
autocorrelation_list = calculate_enhanced_autocorrelation_features(keypoints_data)

back_angle_changes_list1 = calculate_angle_changes(keypoints_data, (6,12,16))
back_angle_changes_list2 = calculate_angle_changes(keypoints_data, (5,11,15))
head_angle_changes_list1 = calculate_angle_changes(keypoints_data, (0,6,12))
head_angle_changes_list2 = calculate_angle_changes(keypoints_data, (0,5,11))
leg_angle_changes_list1 = calculate_angle_changes(keypoints_data, (12,14,16))
leg_angle_changes_list2 = calculate_angle_changes(keypoints_data, (11,13,15))
eye_angle_changes_list1 = calculate_angle_changes(keypoints_data, (1,5,9))
eye_angle_changes_list2 = calculate_angle_changes(keypoints_data, (2,6,10))
strech_angle_changes_list1 = calculate_angle_changes(keypoints_data, (5,8,10))
strech_angle_changes_list2 = calculate_angle_changes(keypoints_data, (6,7,9))
finger_angle_changes_list1 = calculate_angle_changes(keypoints_data, (0,8,10))
finger_angle_changes_list2 = calculate_angle_changes(keypoints_data, (0,7,9))

back_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (6,12,16))
back_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (5,11,15))
head_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (0,6,12))
head_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (0,5,11))
leg_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (12,14,16))
leg_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (11,13,15))
eye_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (1,5,9))
eye_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (2,6,10))
strech_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (5,8,10))
strech_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (6,7,9))
finger_min_angle_changes_list1 = calculate_min_angle_changes(keypoints_data, (0,8,10))
finger_min_angle_changes_list2 = calculate_min_angle_changes(keypoints_data, (0,7,9))

back_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (6,12,16))
back_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (5,11,15))
head_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (0,6,12))
head_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (0,5,11))
leg_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (12,14,16))
leg_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (11,13,15))
eye_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (1,5,9))
eye_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (2,6,10))
strech_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (5,8,10))
strech_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (6,7,9))
finger_max_angle_changes_list1 = calculate_max_angle_changes(keypoints_data, (0,8,10))
finger_max_angle_changes_list2 = calculate_max_angle_changes(keypoints_data, (0,7,9))

# 결과 확인 (예시로 첫 번째 동영상의 평균 키포인트 출력)
# print(angle_changes_list2[0])

In [37]:
features = []
mean_keypoints_all_videos = np.array(mean_keypoints_all_videos)
changes_list = np.array(changes_list)
autocorrelation_list = np.array(autocorrelation_list)

for i in range(len(mean_keypoints_all_videos)):
    combined_feature = np.concatenate([mean_keypoints_all_videos[i].flatten(), changes_list[i].flatten(), autocorrelation_list[i].flatten(),
    # fft_features_list[i].flatten(),
    [back_max_angle_changes_list1[i] - back_min_angle_changes_list1[i],
    back_max_angle_changes_list2[i] - back_min_angle_changes_list2[i],
    head_max_angle_changes_list1[i] - head_min_angle_changes_list1[i],
    head_max_angle_changes_list2[i] - head_min_angle_changes_list2[i],
    leg_max_angle_changes_list1[i] - leg_min_angle_changes_list1[i],
    leg_max_angle_changes_list2[i] - leg_min_angle_changes_list2[i],
    eye_max_angle_changes_list1[i] - eye_min_angle_changes_list1[i],
    eye_max_angle_changes_list2[i] - eye_min_angle_changes_list2[i],
    strech_max_angle_changes_list1[i] - strech_min_angle_changes_list1[i],
    strech_max_angle_changes_list2[i] - strech_min_angle_changes_list2[i],
    finger_max_angle_changes_list1[i] - finger_min_angle_changes_list1[i],
    finger_max_angle_changes_list2[i] - finger_min_angle_changes_list2[i]]])
    
    features.append(combined_feature)


SVM 분류 모델

In [38]:
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np

# 데이터 스케일링
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

def objective(trial):
    # Optuna를 사용한 하이퍼파라미터 탐색 공간 정의
    c = trial.suggest_loguniform('C', 1e-3, 1e3)
    gamma = trial.suggest_loguniform('gamma', 1e-3, 1e3)
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])
    
    # SVM 모델 초기화 및 교차 검증
    svm = SVC(C=c, gamma=gamma, kernel=kernel, random_state=42)
    cv = StratifiedKFold(n_splits=11, shuffle=True, random_state=42)
    scores = cross_val_score(svm, features_scaled, labels, cv=cv, n_jobs=-1)
    
    # 교차 검증 점수의 평균 반환
    return np.mean(scores)

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)  # n_trials: 시도할 횟수

# 최적의 파라미터와 그때의 점수 출력
print(f'Best parameters: {study.best_params}')
print(f'Best cross-validation score: {study.best_value*100:.2f} %')

# 최적의 파라미터로 모델 재학습
best_model = SVC(**study.best_params, random_state=42)
best_model.fit(features_scaled, labels)


[I 2024-03-17 19:37:45,082] A new study created in memory with name: no-name-ab25677c-10ba-446b-820e-632cd434b8d4
/var/folders/t3/mg_pyp2149zcb7zrcn24mdjr0000gn/T/ipykernel_92162/2944653527.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  c = trial.suggest_loguniform('C', 1e-3, 1e3)
/var/folders/t3/mg_pyp2149zcb7zrcn24mdjr0000gn/T/ipykernel_92162/2944653527.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-3, 1e3)
[I 2024-03-17 19:37:46,127] Trial 0 finished with value: 0.929688493324857 and parameters: {'C': 0.11627867313507674, 'gamma': 720.3116361577388, 'kernel': 'poly'}. Best is trial 0 with value: 0.9296884933248

Best parameters: {'C': 75.27641369259489, 'gamma': 0.01186935341318372, 'kernel': 'rbf'}
Best cross-validation score: 97.10 %


SVC(C=75.27641369259489, gamma=0.01186935341318372, random_state=42)